In [1]:
import pandas as pd
import numpy as np
import os
from scipy.cluster import hierarchy as hc
import matplotlib
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from scipy import stats
import math

%matplotlib inline
warnings.filterwarnings(action='ignore')

matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

# Load data

In [2]:
# dir_root = 'C:/Users/kkt71/OneDrive/바탕 화면/기업가치분석'
dir_root = 'C:/Users/10188/Desktop/기업가치/상관분석/기업가치분석_data_result'
dir_master = os.path.join(dir_root, '01. data', 'master')
dir_processed = os.path.join(dir_root, '01. data', 'processed')
dir_result = os.path.join(dir_root, '03. result')

In [3]:
y = '투자비 대비 매출 (Rev/GI)s'
y_file_name = y.replace("/","")
c1 = 'Accenture'
c2 = 'Infosys'
c3 = 'TCS'
c4 = 'Cognizant'

In [4]:
outperform = [c1, c2, c3]
lowperform = [c4]

In [5]:
# dir_result_all = os.path.join(dir_result, 'All_ITS', y)
dir_result_all = os.path.join(dir_result, 'All_ITS', y_file_name)

if os.path.exists(dir_result_all)==False:
    os.makedirs(dir_result_all)

# total 기간에 대해서, 존재하는 변수 기준으로 inner join

In [6]:
df_c1 = pd.read_csv(os.path.join(dir_result, c1, y_file_name, y_file_name+'.csv')).iloc[:,:5]
df_c2 = pd.read_csv(os.path.join(dir_result, c2, y_file_name, y_file_name+'.csv')).iloc[:,:5]
df_c3 = pd.read_csv(os.path.join(dir_result, c3, y_file_name, y_file_name+'.csv')).iloc[:,:5]
df_c4 = pd.read_csv(os.path.join(dir_result, c4, y_file_name, y_file_name+'.csv')).iloc[:,:5]

In [7]:
df_c1.rename({'corr' : df_c1.company.unique()[0]}, axis=1, inplace=True)
df_c2.rename({'corr' : df_c2.company.unique()[0]}, axis=1, inplace=True)
df_c3.rename({'corr' : df_c3.company.unique()[0]}, axis=1, inplace=True)
df_c4.rename({'corr' : df_c4.company.unique()[0]}, axis=1, inplace=True)

df_c1.drop(['company'], axis=1, inplace=True)
df_c2.drop(['company'], axis=1, inplace=True)
df_c3.drop(['company'], axis=1, inplace=True)
df_c4.drop(['company'], axis=1, inplace=True)

In [8]:
df_all_variable = pd.merge(pd.merge(pd.merge(df_c1, df_c2, on=['variable','group', 'importance'], how='inner'), 
                          df_c3, on=['variable','group', 'importance'], how='inner'), 
                 df_c4, on=['variable','group', 'importance'], how='inner')

In [9]:
df_all_variable = df_all_variable[['variable', 'group', 'importance']+outperform+lowperform]

# 0.7 이상 상관계수가 하나라도 있으면, 남기도록 outer join

In [10]:
df_c1 = pd.read_csv(os.path.join(dir_result, c1, y_file_name, 'compare_period_corr_07.csv'))
df_c2 = pd.read_csv(os.path.join(dir_result, c2, y_file_name, 'compare_period_corr_07.csv'))
df_c3 = pd.read_csv(os.path.join(dir_result, c3, y_file_name, 'compare_period_corr_07.csv'))
df_c4 = pd.read_csv(os.path.join(dir_result, c4, y_file_name, 'compare_period_corr_07.csv'))

In [11]:
# df_c1.rename({'corr_total' : df_c1.company.unique()[0]}, axis=1, inplace=True)
df_c2.rename({'corr_total' : df_c2.company.unique()[0]}, axis=1, inplace=True)
df_c3.rename({'corr_total' : df_c3.company.unique()[0]}, axis=1, inplace=True)
df_c4.rename({'corr_total' : df_c4.company.unique()[0]}, axis=1, inplace=True)

# df_c1.drop(['corr_sub', 'company'], axis=1, inplace=True)
df_c2.drop(['corr_sub', 'company'], axis=1, inplace=True)
df_c3.drop(['corr_sub', 'company'], axis=1, inplace=True)
df_c4.drop(['corr_sub', 'company'], axis=1, inplace=True)

In [12]:
# df_c1 = df_c1.dropna()
df_c2 = df_c2.dropna()
df_c3 = df_c3.dropna()
df_c4 = df_c4.dropna()

In [13]:
# df_all = pd.merge(pd.merge(df_c1, df_c3, on=['variable','group', 'importance'], how='outer'), 
#                           df_c4, on=['variable','group', 'importance'], how='outer')
                 

In [14]:
df_all = pd.merge(pd.merge(pd.merge(df_c1, df_c2, on=['variable','group', 'importance'], how='outer'), 
                          df_c3, on=['variable','group', 'importance'], how='outer'), 
                 df_c4, on=['variable','group', 'importance'], how='outer')
# df_all = pd.merge(pd.merge(df_c2, df_c3, on=['variable','group', 'importance'], how='outer'), 
#                           df_c4, on=['variable','group', 'importance'], how='outer')

In [15]:
df_all = df_all[['variable', 'group', 'importance']+list(set(outperform)&set(df_all.columns))+list(set(lowperform)&set(df_all.columns))]

In [16]:
# df_all = df_all[['variable', 'group', 'importance']+[c1, c3, c4]]

# 위의 두 개 inner join

In [17]:
df_all_merged = pd.merge(df_all, df_all_variable, how='inner', on=['variable','group','importance'], suffixes = ("_filtered", "_show"))

In [18]:
# df_all_merged.to_csv(os.path.join(dir_result, 'All_ITS', 'corr_y_x_over7.csv'), index=False, encoding='utf-8-sig')

In [19]:
writer = pd.ExcelWriter(os.path.join(dir_result_all, 'corr_y_x_over7.xlsx'), engine='xlsxwriter')
df_all_merged.to_excel(writer, index=False)
writer.save()

In [20]:
df_all_merged.columns

Index(['variable', 'group', 'importance', 'Infosys_filtered', 'TCS_filtered',
       'Cognizant_filtered', 'Accenture', 'Infosys_show', 'TCS_show',
       'Cognizant_show'],
      dtype='object')

In [21]:
df_all_merged

,variable,group,importance,Infosys_filtered,TCS_filtered,Cognizant_filtered,Accenture,Infosys_show,TCS_show,Cognizant_show
0,CF_Sale/Maturity of Investment,CF,5.0,NaN,NaN,NaN,0.706,-0.347,0.138,-0.275
1,ROGI(%),수익성,0.0,0.787,0.796,0.940,-0.204,0.787,0.796,0.940
2,무형자산회전율s,활동성,0.0,0.782,0.702,0.789,0.042,0.782,0.702,0.789
3,FS_Working Capital to Total Assets,FS,5.0,0.703,NaN,NaN,0.018,0.703,0.624,0.613
4,비유동비율(천원),안정성,2.0,-0.740,NaN,NaN,0.165,-0.740,-0.698,-0.671
...,...,...,...,...,...,...,...,...,...,...
68,BS_Net Debt Incl. Pref.Stock & Min.Interest,BS,5.0,NaN,-0.703,NaN,0.320,-0.325,-0.703,-0.061
69,순부채(천원),안정성,2.0,NaN,-0.703,NaN,0.320,-0.325,-0.703,-0.061
70,BS_Additional Paid-In Capital,BS,5.0,NaN,-0.711,NaN,-0.442,0.622,-0.711,0.684
71,"BS_Accumulated Depreciation, Total",BS,5.0,NaN,-0.743,NaN,0.217,0.403,-0.743,0.451
